In [4]:
import http.server # 가상의 웹 서버를 만들 수 있는 모듈
import urllib.parse # url 주소를 parsing해주는 모듈
import os           # 현재 디렉터리에 접근할 수 있는 모듈

class SimpleBoard:
    def __init__(self, filename="board.txt"):
        self.posts = {}
        self.next_id = 1 # 다음 순서로 추가될 키
        self.filename = filename
        self.load_posts()

    def load_posts(self):
        if os.path.exists(self.filename):
            with open(self.filename, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    post_id, title, content = line.strip().split('\t')
                    self.posts[int(post_id)] = {'title': title, 'content':content}
                if self.posts: # 비어 있는 딕셔너리가 아니면 True
                    self.next_id = max(self.posts.keys()) + 1
    def save_posts(self):
        with open(self.filename, 'w') as file:
            print("saving...")
            for post_id, post in self.posts.items():
                file.write(f"{post_id}\t{post['title']}\t{post['content']}\n")

    def list_posts(self):
        posts_html = ""
        for pid, post in self.posts.items():
            posts_html += (
                f"<tr>"
                f"<td>{pid}</td>"
                f"<td>{post['title']}</td>"
                f"<td>{post['content']}</td>"
                f"<td><a href='/delete?id={pid}'>Delete</a></td>"
                f"</tr>"
            )
        return posts_html

    def add_post(self, title, content):
        self.posts[self.next_id] = {'title': title, 'content': content}
        self.next_id += 1
        self.save_posts()

    def delete_post(self, post_id):
        if post_id in self.posts:
            del self.posts[post_id]
            self.save_posts()


board = SimpleBoard()

class RequestHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        path, _, query_string = self.path.partition("?")
        # shopping.naver.com/search/all?query=코일매트
        # ?를 기준으로 우측이 query_string(HTTP url 요청 시 추가로 들어가는 parameter)
        # 도메인 이름을 제외한 /search/all은 page 이름 = 웹 페이지 내부 경로
        # 예시를 들자면 path = '/search/all', query_string = 'query=코일매트'
        query = urllib.parse.parse_qs(query_string)
        # query_string은 다시 parsing됨: {'query': ['코일매트']}

        if path == '/':  # 도메인 이름에 해당하는 메인 페이지
            self.list_page()
        elif path == '/add':
            self.add_page()
        elif path == '/delete':
            self.delete_post(query)

    def do_POST(self):
        path, _, query_string = self.path.partition("?")
        query = urllib.parse.parse_qs(query_string)
        content_length = int(self.headers['Content-Length'])
        read_content = self.rfile.read(content_length)
        post_data = urllib.parse.parse_qs(read_content.decode('utf-8'))
        print("post_data", post_data)

        if path == '/add':
            self.add_post(post_data)
    def add_page(self):
        self.send_response(200) # 상태코드 : 200 - Success
        self.send_header('Content-type', 'text/html') # body의 문서 종류
        self.end_headers()

        html = f"""
            <html>
                <body>
                    <h1> Add Post </h1>
                    <form action="/add" method="POST">
                        Title : <input type="text" name="title"> <br>
                        Content :
                        <textarea name="content"></textarea> <br>
                        <input type="submit" value="Add"> <br>
                    </form>
                    <br> <a href="/">Back to List</a>
                </body>
            </html>
        """

        self.wfile.write(html.encode('utf-8')) # body에 html 태그 내용을 태워서 보냄
    def list_page(self):
        self.send_response(200) # 상태코드 : 200 - Success
        self.send_header('Content-type', 'text/html') # body의 문서 종류
        self.end_headers()

        html = f"""
            <html>
                <body>
                    <h1> Simple Board</h1>
                    <table border="1">
                        <tr>
                            <th> ID </th>
                            <th> Title </th>
                            <th> Content </th>
                            <th> Delete </th>
                        </tr>
                        {board.list_posts()}
                    </table>
                    <br> <a href="/add">Add Post</a>
                </body>
            </html>
        """

        self.wfile.write(html.encode('utf-8')) # body에 html 태그 내용을 태워서 보냄

    def add_post(self, post_data):
        title = post_data['title'][0]
        content = post_data['content'][0]
        board.add_post(title, content)
        # main page로 다시 되돌아가는 작업
        self.send_response(303)   # 상태코드 303 : redirect
        self.send_header('Location', '/') # main page로 가기
        self.end_headers()

    def delete_post(self, query):
        post_id = query['id'][0]
        post_id = int(post_id)
        board.delete_post(post_id)
        # main page로 다시 되돌아가는 작업
        self.send_response(303)   # 상태코드 303 : redirect
        self.send_header('Location', '/') # main page로 가기
        self.end_headers()


if __name__ == '__main__':
    server = http.server.HTTPServer(('localhost', 8000), RequestHandler)
    print('Starting server at http://localhost:8000')
    server.serve_forever()

OSError: [Errno 98] Address already in use